<a href="https://colab.research.google.com/github/saymynamet/RC-AIStudy/blob/main/Tak's_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
import numpy as np
import random
import os
import scipy.special
from sklearn.model_selection import train_test_split   # ★ 추가

## Load Data & Pre-Processing

In [ ]:
# 1) 데이터 읽기 (인코딩 에러 나면 encoding='latin1' 붙여보기)
df = pd.read_csv('car_purchasing.csv', encoding='latin1')  # 문제 생기면 df = pd.read_csv('car_purchasing.csv', encoding='latin1')

# 2) train / test 분리 (예: 80% / 20%)
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

print(train.head())   # 확인용

            customer name                     customer e-mail  \
249    Wilma L. Stevenson      non.vestibulum@consectetuer.ca   
433               Madeson             libero.et@Crasdictum.ca   
19            Athena Wolf  volutpat.Nulla.facilisis@primis.ca   
322  Livingston, Blaze Y.      montes.nascetur@liberoet.co.uk   
332      Irwin, Dillon J.                 arcu@dictummiac.edu   

                   country  gender        age  annual Salary  \
249  Sao Tome and Principe       0  36.117247    70334.42787   
433       Christmas Island       1  39.814817    56086.45033   
19                    Iraq       0  58.741842    55368.23716   
322                 Serbia       0  60.180119    58837.97088   
332                Tokelau       0  31.244838    41361.95045   

     credit card debt    net worth  car purchase amount  
249       9823.218967  632600.4718          46135.27233  
433       6588.606462  228388.5491          29519.56184  
19       13272.946470  566022.1306          54827.

In [ ]:
# 3) 입력 컬럼 / 타깃 컬럼 선택
# 이름, 이메일, 국가 같은 건 버리고 수치형만 사용
feature_cols = ['gender', 'age', 'annual Salary', 'credit card debt', 'net worth']
target_col = 'car purchase amount'

x_train = train[feature_cols]
x_test  = test[feature_cols]

y_train = train[target_col].values
y_test  = test[target_col].values   # 나중에 평가용 (필수는 아님)

In [ ]:
# 4) PyTorch 텐서로 변환 + 표준화(입력만)
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_test_tensor  = torch.tensor(x_test.values,  dtype=torch.float32)

mean = x_train_tensor.mean(dim=0, keepdim=True)
std  = x_train_tensor.std(dim=0, keepdim=True)

x_train_tensor = (x_train_tensor - mean) / std
x_test_tensor  = (x_test_tensor  - mean) / std

In [ ]:
# 타깃도 스케일 맞춰주는 게 좋음 (0~1로 정규화)
y_min = y_train.min()
y_max = y_train.max()
y_train_norm = (y_train - y_min) / (y_max - y_min)   # 0~1로

# 학습할 땐 이걸 사용해야 함!!
target = y_train_norm[i]
n.train(inputs, target)

In [ ]:
output_norm = n.query(inputs)[0, 0]          # 0~1 가정
pred = output_norm * (y_max - y_min) + y_min # 원래 금액

## Train

In [ ]:
#neural network class definition
class neuralNetwork:

    #initialize the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #가중치 행렬 wih(input_hidden)와 who(hidden_output)
        #배열 내 가중치는 w_i_j로 표기. 노드 i에서 다음 계층의 노드 j로 연결됨을 의미
        #w11 w21
        #w12 w22

        self.wih = np.random.normal(0.0, pow(self.hnodes, - 0.5),(self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, - 0.5),(self.onodes, self.hnodes))

        #learning rate
        self.lr = learningrate

        #expit()는 시그모이드 함수
        self.activation_function = lambda x: scipy.special.expit(x)

        pass

    #신경망 학습시키기
    def train(self, inputs_list, targets_list):
      #입력 리스트를 2차원의 행렬로 변환
      inputs = np.array(inputs_list, ndmin=2).T
      targets = np.array(targets_list, ndmin=2).T
# 순전파
      #은닉 계층으로 들어오는 신호를 계산
      hidden_inputs = np.dot(self.wih, inputs)
      #은닉 계층에서 나가는 신호를 계산
      hidden_outputs = self.activation_function(hidden_inputs)

      #최종 출력 계층으로 들어오는 신호를 계산
      final_inputs = np.dot(self.who, hidden_outputs)
      #최종 출력 계층에서 나가는 신호를 계산
      final_outputs = self.activation_function(final_inputs)

# 역전파
      #출력 계층의 오차는 (실제 값 - 계산 값)
      output_errors = targets - final_outputs
      #은닉 계층의 오차는 가중치에 의해 나뉜 계층의 오차들을 재조합해 계산
      hidden_errors = np.dot(self.who.T, output_errors)
# 가중치 업데이트
      #은닉 계층과 출력 계층 간의 가중치 업데이트
      self.who += self.lr*np.dot((output_errors*final_outputs*(1.0-final_outputs)), np.transpose(hidden_outputs))

      #입력 계층과 은닉 계층 간의 가중치 업데이트
      self.wih += self.lr*np.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)), np.transpose(inputs))
      pass


    #query the neural network
    def query(self, inputs_list):
      #입력 리스트를 2차원 행렬로 변환
      inputs = np.array(inputs_list, ndmin=2).T
      #은닉 계층으로 들어오는 신호를 계산
      #x_hidden = w_hidden x I (x는 가중치 행렬, I는 입력 값들의 행렬)
      #입력 계층과 은닉 계층 사이의 가중치 행렬은 입력 핼렬과 조합됨
      hidden_inputs = np.dot(self.wih, inputs)
      #은닉 계층에서 나가는 신호를 계산
      hidden_outputs = self.activation_function(hidden_inputs)
      #최종 출력 계층으로 들어오는 신호 계산
      final_inputs = np.dot(self.who, hidden_outputs)
      #최종 출력 계층에서 나가는 신호 계산
      final_outputs = self.activation_function(final_inputs)

      return final_outputs

In [ ]:
# 6) 네트워크 초기화
input_nodes  = x_train_tensor.shape[1]  # 특징 개수(5개)
hidden_nodes = 16
output_nodes = 1                        # 구매 금액 1개 출력
learning_rate = 0.01

n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [ ]:
# 7) 학습 루프
epochs = 10
for e in range(epochs):
    for i in range(len(x_train_tensor)):
        inputs = x_train_tensor[i].numpy()
        target = y_train_norm[i]    # 0~1로 정규화된 값 사용
        n.train(inputs, target)

    print(f"epoch: {e+1}/{epochs}")


epoch: 1/10
epoch: 2/10
epoch: 3/10
epoch: 4/10
epoch: 5/10
epoch: 6/10
epoch: 7/10
epoch: 8/10
epoch: 9/10
epoch: 10/10


In [ ]:
# 8) 테스트 데이터에서 예측 예시
test_idx = 0
pred_norm = n.query(x_test_tensor[test_idx].numpy())[0, 0]      # 0~1
pred = pred_norm * (y_max - y_min) + y_min                     # 원래 스케일로 되돌리기
print("예측값:", pred, "/ 실제값:", y_test[test_idx])

예측값: 46429.079569489244 / 실제값: 46082.80993


## Prediction

In [ ]:
import numpy as np
import pandas as pd

preds = []
for i in range(len(x_test_tensor)):
    # 표준화된 입력 사용
    inputs = x_test_tensor[i].numpy()          # (특징 수,)

    # 신경망 예측 (0~1 범위의 값)
    output_norm = n.query(inputs)[0, 0]        # shape (1,1) → 스칼라

    # 원래 car purchase amount 스케일로 되돌리기
    pred = output_norm * (y_max - y_min) + y_min
    preds.append(pred)

# test 데이터프레임에 예측값 붙이기
result = test.copy()
result['predicted_car_purchase_amount'] = preds

# CSV로 저장
result.to_csv('baseline_submit.csv', index=False)
print("저장 완료: baseline_submit.csv")

저장 완료: baseline_submit.csv




```
# 코드로 형식 지정됨
```

## Submission